In [2]:
import pickle
from loadData import dataLoader
import tensorflow as tf
from CalculateEER import SKGetEER as GetEER

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
data = pickle.load(open("./Data/dataBenchmarkTf.pickle", "rb"))

X = data['data']
Y = data['lables']
X = X.reshape(20400, 1, 31, 1)

data['data'] = X
pickle.dump(data, open("./Data/cnnkeystroke.pickle", "wb"))

data = dataLoader("./Data/cnnkeystroke.pickle")
print(data.data.shape, data.data_labels.shape)
print(data.train.shape, data.test.shape, data.valid.shape)

Loading Data...
Dividing Data...
14280 3060 3060
Divided data.
(20400, 1, 31, 1) (20400, 51)
(14280, 1, 31, 1) (3060, 1, 31, 1) (3060, 1, 31, 1)


In [5]:
inp = tf.placeholder(tf.float32, shape=[None, 1, 31, 1])
out = tf.placeholder(tf.float32, shape = [None, 51])


def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

W_conv1 = weight_variable([1, 5, 1, 32])
b_conv1 = bias_variable([32])

h_conv1 = tf.nn.relu(conv2d(inp, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([1, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

print(h_pool2.shape)
W_fc1 = weight_variable([1 * 8 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 1*8*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 51])
b_fc2 = bias_variable([51])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
y_ = tf.nn.softmax(y_conv)

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=out, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(out, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

(?, 1, 8, 64)


In [6]:
with tf.Session() as sess:
    try:
        sess.run(tf.global_variables_initializer())
        batchSize = 128
        for epoch in range(500):
            j = -1
            i = 0
            while i < data.train.shape[0]:
                j += 1
                batch = data.train[i: i + batchSize]
                batch_labels = data.train_labels[i: i + batchSize]
                i = i + batchSize
                #print(batch.shape)
                train_step.run(feed_dict={inp: batch, out: batch_labels, keep_prob: 0.5})
                if not j%50:
                    train_accuracy = accuracy.eval(feed_dict={inp: batch, out: batch_labels, keep_prob: 1.0})
                    train_out = sess.run(y_, feed_dict={inp: batch, out: batch_labels, keep_prob: 1.0})
                    valid_accuracy = accuracy.eval(feed_dict={inp: data.valid, out: data.valid_labels, keep_prob: 1.0})
                    valid_out = sess.run(y_, feed_dict={inp: data.valid, out: data.valid_labels, keep_prob: 1.0})
                    #print(valid_out[0])
                    test_accuracy = accuracy.eval(feed_dict={inp: data.test, out: data.test_labels, keep_prob: 1.0})
                    test_out = sess.run(y_, feed_dict={inp: data.test, out: data.test_labels, keep_prob: 1.0})
                    TEER = GetEER(test_out, data.test_labels)
                    VEER = GetEER(valid_out, data.valid_labels)
                    print(epoch, j, train_accuracy, valid_accuracy, test_accuracy, VEER, TEER)
    except:
        d = {}
        d['cnn'] = sess.run(y_, feed_dict={inp: data.data, out: data.data_labels, keep_prob: 1.0})
        d['y'] = data.data_labels
        pickle.dump(d, open("./Data/CNNOutput.pickle", "wb"))


0 0 0.046875 0.024509804 0.023202615 0.481951871657754 0.48096192384769537
0 50 0.0703125 0.056862745 0.050326798 0.3370110330992979 0.34082522643408253
